In [3]:
from pykiwoom.kiwoom import *

kiwoom = Kiwoom()
kiwoom.CommConnect(block=True)

kospi = kiwoom.GetCodeListByMarket('0')
kosdaq = kiwoom.GetCodeListByMarket('10')
etf = kiwoom.GetCodeListByMarket('8')

print(len(kospi), kospi)
print(len(kosdaq), kosdaq)
print(len(etf), etf)

AttributeError: 'QAxWidget' object has no attribute 'OnReceiveTrData'

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [166]:
#특정 날짜 23년 2월 2일 크롤링
url = 'http://www.fantastock.co.kr/bbs/board.php?bo_table=free_list23&wr_id=2190#section1'
response = requests.get(
    url,
    headers={
        'User-Agent': 'Mozilla 5.0'
    }
    )
bs = BeautifulSoup(response.text, 'lxml')    

In [167]:
#특정 날짜 23년 2월 2일 크롤링
df = pd.DataFrame(columns=['comment'])
for i in range(len(bs.select('.cmt_contents'))):
    comment = bs.select('.cmt_contents')[i].text
    df.loc[i] = [comment]

In [168]:
df

,comment
0,\n\n \n
1,\n\n \n
2,\n\n \n
3,\n\n \n
4,\n\n전일 미국증시부터 살펴보고 가겠습니다. \n
...,...
81,\n\n\n
82,\n\n\n
83,\n\n \n
84,"\n\n코스피 상승, 코스닥 상승 거래소 : 개인 매도 / 외국인 매수 / 기관 매..."


In [193]:
#게시판에서 각 게시물 주소얻기 
url = 'http://www.fantastock.co.kr/bbs/board.php?bo_table=free_list23'
response = requests.get(
    url,
    headers={
        'User-Agent': 'Mozilla 5.0'
    }
    )
bs = BeautifulSoup(response.text, 'lxml') 

In [198]:
len(bs.select('table>tr'))

33

In [200]:
#1페이지 링크 얻기
for i in range(len(bs.select('table>tr'))):
    date = bs.select('table>tr')[i].select('dd')[1].text
    url_link = bs.select('table>tr')[i].select('a')[0].get('href')

In [204]:
#해당 년도 모든 페이지에 날짜 url 얻기
#마지막 페이지 찾는 작업 필요 
for page_num in range(1,4):
    url = f'http://www.fantastock.co.kr/bbs/board.php?bo_table=free_list22&page={page_num}'
    response = requests.get(
        url,
        headers={
        'User-Agent': 'Mozilla 5.0'
        }
        )
    bs = BeautifulSoup(response.text, 'lxml') 

    for i in range(len(bs.select('table>tr'))):
        date = bs.select('table>tr')[i].select('dd')[1].text
        url_link = bs.select('table>tr')[i].select('a')[0].get('href')
        print(date)

2022.12.30
2022.12.29
2022.12.28
2022.12.27
2022.12.26
2022.12.23
2022.12.22
2022.12.21
2022.12.20
2022.12.19
2022.12.16
2022.12.15
2022.12.14
2022.12.13
2022.12.12
2022.12.09
2022.12.08
2022.12.07
2022.12.06
2022.12.05
2022.12.02
2022.12.01
2022.11.30
2022.11.29
2022.11.28
2022.11.25
2022.11.24
2022.11.23
2022.11.22
2022.11.21
2022.11.18
2022.11.17
2022.11.16
2022.11.15
2022.11.14
2022.11.11
2022.11.10
2022.11.09
2022.11.08
2022.11.07
2022.11.04
2022.11.03
2022.11.02
2022.11.01
2022.10.31
2022.10.28
2022.10.27
2022.10.26
2022.10.25
2022.10.24
2022.10.21
2022.10.20
2022.10.19
2022.10.18
2022.10.17
2022.10.14
2022.10.13
2022.10.12
2022.10.11
2022.10.07
2022.10.06
2022.10.05
2022.10.04
2022.09.30
2022.09.29
2022.09.28
2022.09.27
2022.09.26
2022.09.23
2022.09.22
2022.09.21
2022.09.20
2022.09.19
2022.09.16
2022.09.15
2022.09.14
2022.09.13
2022.09.08
2022.09.07
2022.09.06
2022.09.05
2022.09.02
2022.09.01
2022.08.31
2022.08.30
2022.08.29
2022.08.26
2022.08.25
2022.08.24
2022.08.23
2022.08.22

In [203]:
url

'http://www.fantastock.co.kr/bbs/board.php?bo_table=free_list22&page=4'

In [181]:
#모든 년도 모든 페이지에 날짜 url 얻기
#마지막 페이지 찾는 작업 필요 
for year in range(22,24):
    for page_num in range(1,4):
        url = f'http://www.fantastock.co.kr/bbs/board.php?bo_table=free_list{year}&page={page_num}'
        response = requests.get(
            url,
            headers={
            'User-Agent': 'Mozilla 5.0'
            }
            )
        bs = BeautifulSoup(response.text, 'lxml') 

        for i in range(33):
            date = bs.select('table>tr')[i].select('dd')[1].text
            url_link = bs.select('table>tr')[i].select('a')[0].get('href')
            print(date, url_link)

2022.12.30 http://www.fantastock.co.kr/bbs/board.php?bo_table=free_list22&wr_id=37528&page=1#section1
2022.12.29 http://www.fantastock.co.kr/bbs/board.php?bo_table=free_list22&wr_id=37462&page=1#section1
2022.12.28 http://www.fantastock.co.kr/bbs/board.php?bo_table=free_list22&wr_id=37395&page=1#section1
2022.12.27 http://www.fantastock.co.kr/bbs/board.php?bo_table=free_list22&wr_id=37333&page=1#section1
2022.12.26 http://www.fantastock.co.kr/bbs/board.php?bo_table=free_list22&wr_id=37278&page=1#section1
2022.12.23 http://www.fantastock.co.kr/bbs/board.php?bo_table=free_list22&wr_id=37214&page=1#section1
2022.12.22 http://www.fantastock.co.kr/bbs/board.php?bo_table=free_list22&wr_id=37143&page=1#section1
2022.12.21 http://www.fantastock.co.kr/bbs/board.php?bo_table=free_list22&wr_id=37054&page=1#section1
2022.12.20 http://www.fantastock.co.kr/bbs/board.php?bo_table=free_list22&wr_id=36988&page=1#section1
2022.12.19 http://www.fantastock.co.kr/bbs/board.php?bo_table=free_list22&wr_id=36

IndexError: list index out of range

In [186]:
url


'http://www.fantastock.co.kr/bbs/board.php?bo_table=free_list23&page=2'